# Generamos Foto Previsión desde Fotos Histórico

_Importación de librerias_

In [6]:
import pandas as pd
import tfm_helpers as tfm

pd.set_option("display.max_columns", None)

Procesamos los ficheros para generar previsiones agrupadas por año

In [7]:
year = "2099"
file = r"data\processed\snapshots\reservas\parquet\2007\07\20070701_reservas.parquet"


In [15]:

current_snapshot_data = pd.read_parquet(file, columns=["Hotel", "Fecha venta", "Fecha entrada", "Fecha salida", "Noches", "Cantidad habitaciones", "Estado reserva", "Fecha Borrado", "Días Anticipación", "Fecha foto"])
current_snapshot_data.reset_index(inplace=True)
current_snapshot_data["Hotel"] = current_snapshot_data.Hotel.astype("category")
current_snapshot_data["Estado reserva"] = current_snapshot_data["Estado reserva"].astype("category")
current_snapshot_data.memory_usage(deep=True)

filtro_reservas_activas = ((current_snapshot_data["Estado reserva"] == "Reserva") | (current_snapshot_data["Estado reserva"] == "Entrada"))
current_snapshot_data = current_snapshot_data[filtro_reservas_activas]
 
fecha_foto = current_snapshot_data["Fecha foto"].min()

current_snapshot_data["Fecha inicio"] = current_snapshot_data["Fecha entrada"].apply(lambda x : x if x > fecha_foto else fecha_foto)

current_snapshot_data["Fecha prevision"] = [pd.date_range(entrada, salida, freq="d", closed="left") for entrada, salida in 
                                                zip(pd.to_datetime(current_snapshot_data["Fecha inicio"]), 
                                                    pd.to_datetime(current_snapshot_data["Fecha salida"]))]
m

current_snapshot_data = current_snapshot_data.explode("Fecha prevision")
 

In [16]:
current_snapshot_data[current_snapshot_data["Id Reserva"] == "04-2007-2009-1"].head(50)

,Id Reserva,Hotel,Fecha venta,Fecha entrada,Fecha salida,Noches,Cantidad habitaciones,Estado reserva,Fecha Borrado,Días Anticipación,Fecha foto,Fecha prevision
6,04-2007-2009-1,04,2007-05-19,2007-06-26,2007-07-03,7,1,Entrada,1753-01-01,38,2007-07-01,2007-07-01
6,04-2007-2009-1,04,2007-05-19,2007-06-26,2007-07-03,7,1,Entrada,1753-01-01,38,2007-07-01,2007-07-02


In [ ]:

result_ocupacion_data = pd.DataFrame()
result_ocupacion_data = pd.concat([result_ocupacion_data, current_snapshot_data]) 

result_ocupacion_data.set_index(["Hotel", "Fecha prevision", "Fecha foto"], inplace=True)
result_ocupacion_data.sort_index(inplace=True)


In [ ]:
result_ocupacion_data.info(memory_usage="deep")


In [ ]:
# result_ocupacion_data.to_parquet(tfm.getPathToProcessedSnapshotOcupacion("parquet", year), index=True)
# result_ocupacion_data.to_csv(tfm.getPathToProcessedSnapshotOcupacion("csv", year), index=True, sep="|", escapechar="~", encoding="cp1252")